 # Load Data

In [1]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset, Experiment

ws = Workspace.from_config()

experiment_name = "heart_predictions"
experiment = Experiment(ws, experiment_name)

dataset = Dataset.get_by_name(ws, name='heart')
dataset.to_pandas_dataframe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


 # Create Compute

In [27]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


 # Automl Config

In [5]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task="classification", 
compute_target=compute_target,
training_data=dataset,
label_column_name="output",
n_cross_validations=5,
experiment_timeout_hours=0.5)

 # Train

In [6]:
run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart_predictions,AutoML_e239ac64-cc3a-4f5b-b6fc-4a8d21935221,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [2]:
from azureml.core import Run
from azureml.train.automl.run import AutoMLRun
run = AutoMLRun(experiment, run_id='AutoML_e239ac64-cc3a-4f5b-b6fc-4a8d21935221')

In [3]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [4]:
run.wait_for_completion()

{'runId': 'AutoML_e239ac64-cc3a-4f5b-b6fc-4a8d21935221',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2022-03-12T21:10:18.978799Z',
 'endTimeUtc': '2022-03-12T21:53:52.979853Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"heart_predictions","subscription_id":"67021087-2c78-4338-bc3d-dd21237069f1","resource_group":"analytics-mlops-rg","workspace_name":"analytics-AML-WS","region":"westeurope","compute_target":"auto-ml","spark_service":null,"azu

 # Get Best Model

In [5]:
from azureml.core import Model
best_ml_run = run.get_best_child()
best_ml_run.get_details()

{'runId': 'AutoML_e239ac64-cc3a-4f5b-b6fc-4a8d21935221_30',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2022-03-12T21:52:53.709037Z',
 'endTimeUtc': '2022-03-12T21:52:57.373661Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'heart_predictions\',\'compute_target\':\'auto-ml\',\'subscription_id\':\'67021087-2c78-4338-bc3d-dd21237069f1\',\'region\':\'westeurope\',\'spark_service\':None}","ensemble_run_id":"AutoML_e239ac64-cc3a-4f5b-b6fc-4a8d21935221_30","experiment_name":"heart_predictions","workspace_name":"analytics-AML-WS","subscription_id":"67021087-2c78-4338-b

In [6]:
best_model = best_ml_run.register_model("heart-automl", model_path=best_ml_run.get_details()["properties"]["model_output_path"])
best_model

Model(workspace=Workspace.create(name='analytics-aml-ws', subscription_id='67021087-2c78-4338-bc3d-dd21237069f1', resource_group='analytics-mlops-rg'), name=heart-automl, id=heart-automl:6, version=6, tags={}, properties={})

 # Inference Config

In [7]:
import os
source_dir_name = "source_dir_automl"
source_dir = f"./{source_dir_name}"
os.makedirs(source_dir, exist_ok=True)
best_ml_run.download_file("outputs/scoring_file_v_1_0_0.py",output_file_path=source_dir)
best_ml_run.download_file("outputs/conda_env_v_1_0_0.yml",output_file_path=source_dir)

In [8]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification("heart_env","./source_dir_automl/conda_env_v_1_0_0.yml")
env.inferencing_stack_version='latest'
inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir_automl",
    entry_script="./scoring_file_v_1_0_0.py",
)

 # Deploy Webservice

In [9]:
from azureml.core.webservice import AciWebservice, Webservice
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = 1,
    memory_gb = 1,
    auth_enabled=True,
    enable_app_insights=True)

In [10]:
service = Model.deploy(
    ws,
    "heart-predict",
    [best_model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-13 20:44:38+00:00 Creating Container Registry if not exists.
2022-03-13 20:44:38+00:00 Registering the environment.
2022-03-13 20:44:39+00:00 Use the existing image.
2022-03-13 20:44:39+00:00 Generating deployment configuration.
2022-03-13 20:44:40+00:00 Submitting deployment to compute.
2022-03-13 20:44:45+00:00 Checking the status of deployment heart-predict..
2022-03-13 20:47:15+00:00 Checking the status of inference endpoint heart-predict.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [11]:
logs = service.get_logs()
logs

'2022-03-13T20:47:05,135814000+00:00 - gunicorn/run \n\nPATH environment variable: /azureml-envs/azureml_4943656157e05766fedf432d8be562a4/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin\nPYTHONPATH environment variable: \n\nPip Dependencies\n---------------\n2022-03-13T20:47:05,136986700+00:00 - iot-server/run \n2022-03-13T20:47:05,135792900+00:00 - rsyslog/run \n2022-03-13T20:47:05,189217700+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_4943656157e05766fedf432d8be562a4/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2022-03-13T20:47:05,549354700+00:00 - iot-server/finish 1 0\n2022-03-13T20:47:05,557016000+00:00 - Exit code 1 is normal. Not restarting iot-server.\nadal==1.2.7\napplicationinsights==0.11.10\nargcomplete==1.12.3\nazure-common==1.1.28\nazure-core==1.23.0\nazure-graphrbac==0.61.1\nazure-identity==1.7.0\nazure-mgmt-authorization==0.

 # Use Webservice

In [12]:
import json
data = {
  "data": [
    {
      "age": 20,
      "sex": 1,
      "cp": 0,
      "trtbps": 0,
      "chol": 0,
      "fbs": 0,
      "restecg": 0,
      "thalachh": 0,
      "exng": 0,
      "oldpeak": 0,
      "slp": 0,
      "caa": 0,
      "thall": 0
    }
  ],
  "method": "predict"
}
input_data = json.dumps(data)

In [13]:
response = service.run(input_data)
response

'{"result": [0]}'

In [14]:
import requests

scoring_uri = service.scoring_uri
key = service.get_keys()[0]
headers = {"Content-Type": "application/json",
"Authorization": f"Bearer {key}"}
response = requests.post(scoring_uri, input_data, headers=headers)
print(response)
print(response.json())

<Response [200]>
{"result": [0]}


 # Cleanup
 

In [28]:
from azureml.exceptions import ComputeTargetException
from azureml.core import ComputeTarget
try:
    compute = ComputeTarget(ws, amlcompute_cluster_name)
    compute.delete()
    print("Deleted cluster")
except ComputeTargetException:
    print("Cluster allready deleted")

Deleted cluster


In [29]:
from azureml.core.webservice import Webservice
Webservice.delete(service)

Current provisioning state of AmlCompute is "Deleting"

